In [16]:
class Featurizer():
    def __init__(self):
        super(Featurizer, self).__init__()
        self.audio_dir = "data/recordings/"
        
        self.direction = Direction()
        
    def load_data(self, path):
    
        if os.path.exists(path):
            return pd.read_csv(path, index_col=False)

        else:
            header = 'filename location_id device_id user_id coordinate_x coordinate_y coordinate_z direction\
            spec_centroid spec_rolloff spec_bandwidth rmse chroma tempogram max_amplitude min_amplitude \
            mean_amplitude standard_deviation variance skew kurtosis zero_crossing_rate quantiles_10 quantiles_25 \
            quantiles_50 quantiles_75'

            for i in range(1, 20):
                header += f' mfcc{i}'
            header = header.split()

            file = open(path, 'w', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(header)

            return pd.read_csv(path, index_col=False)
        
    def set_utils(self, location_id, user_id):
        self.location_id = location_id
        self.user_id = user_id
        
        utils = Utils()
        self.address = utils.user[user_id]['address']
        self.uuid = utils.user[user_id]['uuid']
        self.tag = utils.user[user_id]['tag']

    def get_metadata(self):
        metadata = {'location_id': self.location_id,
                    'device_id': self.tag,
                    'user_id': self.user_id
                   }
        metadata = pd.DataFrame([list(metadata.values())], columns = list(metadata.keys()))
        return metadata

    def get_location(self):
        
        localizer = Localizer()
        localizer.connect_serial()

        while True:
            try:
                data = localizer.get_serial_data()
                localizer.compute_coordinates(data)
                localizer.sanitize_coordinates()

                for coord in localizer.coords:
                    if coord["id"]==self.tag:
                        location = {}
                        location['coordinate_x'] = coord["x"]
                        location['coordinate_y'] = coord["y"]
                        location['coordinate_z'] = coord["z"]

                        location = pd.DataFrame([list(location.values())], columns = list(location.keys()))

                        localizer.disconnect_serial()
                        
                        print("Received location: \n", location, "\n")
                        return location
            except Exception as e:
                continue
      
    def get_direction(self):
        if not self.direction.is_connected():
            self.direction.pair(self.address)
        
        dr = self.direction.get_data(self.uuid)
        dr = {'direction': dr}
        dr = pd.DataFrame([list(dr.values())], columns = list(dr.keys()))
        print("Received direction: \n", dr, "\n")

        return dr
    
    def close_connections(self):
        self.direction.unpair()
    
    def get_audio_features(self, i, runtime):
        audio_file = "record_" + f"{i:04d}" + ".wav"
        audio_path = self.audio_dir + audio_file

        audio = Audio()
        audio.initiate_mic()
        audio.record_audio(audio_path, runtime)
        audio.close_mic()
        audio.save_audio() 
        audio.load_audio()

        audio_feat = audio.create_audio_features()
        audio_feat_df = pd.DataFrame([list(audio_feat.values())], columns = list(audio_feat.keys()))
        audio_feat_df.insert(loc=0, column='filename', value=[audio_file])

        return audio_feat_df
    
    def get_rf_attributes(self):
        pass